In [1]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances

In [2]:
data_train = pd.read_csv('TopicModeling/data/data_train.csv', usecols=['title','summarize','content','created','url','category','tag','source'])
data_train.head()

,title,summarize,content,created,url,category,tag,source
0,Phát động cuộc thi tiếng Anh TOEFL Challenge d...,Tổ chức Giáo dục IIG Việt Nam phát động cuộc t...,quang_cảnh phát_động tổ_chức thi ủng_hộ gdđt g...,20/09/19 10:27 GMT+7,https://baomoi.com/c/32258593.epi,Giáo dục,TOEFL Primary Challenge TOEFL Primary TOEFL Ju...,GD&TĐ
1,Hải Phòng: 4 cháu bé nhập viện cấp cứu sau khi...,Sau khi chia nhau uống chai sữa hoa quả do 1 n...,bác_sĩ phạm thị thành giám_đốc bvđk chân xác_n...,17/09/19 22:00 GMT+7,https://baomoi.com/c/32226941.epi,Đời sống,Chai sữa H.A. Công an phường Lạch Tray Ngộ độc...,GĐ&XH
2,"Văn Hậu được Heerenveen đăng ký lần đầu, có th...",Hậu vệ người Việt Nam ngồi ghế dự bị Heerenvee...,chính_thức tập_luyện heerenveen văn hậu chính_...,22/09/19 16:50 GMT+7,https://baomoi.com/c/32283331.epi,Thể thao,Lucas Woudenberg Văn Hậu Dự bị Hậu vệ Johny Ja...,VTC
3,Anh Nguyễn Tuấn Nam tái đắc cử Chủ tịch Hội LH...,"Sáng nay (20/9), Đại hội đại biểu Hội LHTNVN t...",đoàn chủ_tịch đại_hội lhtn tỉnh lạng sơn ảnh c...,20/09/19 11:06 GMT+7,https://baomoi.com/c/32259106.epi,Xã hội,Nguyễn Tuấn Nam Hội LHTNVN Nguyễn Kim Quy Hồ P...,Tiền Phong
4,Trồng hoa kiểng trong chậu phù hợp với nông ng...,Trung tâm Khuyến nông TPHCM vừa triển khai mô ...,mô_hình triển_khai hoa dừa cạn yến thảo nguồn_...,23/09/19 09:49 GMT+7,https://baomoi.com/c/32289518.epi,Kinh tế,Trung tâm Khuyến nông TPHCM Hoa kiểng Dừa cạn ...,SGGP


In [3]:
data_train.dropna(subset=['content'], inplace=True)

In [4]:
tfidf_title_vectorizer = TfidfVectorizer()
tfidf_title_features = tfidf_title_vectorizer.fit_transform(data_train['content'])

In [11]:
def tfidf_based_model(row_index, num_similar_items):
    couple_dist = pairwise_distances(tfidf_title_features, tfidf_title_features[row_index])
    print(couple_dist.ravel())
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    print(indices)
    df = pd.DataFrame({'publish_date': data_train['created'][indices].values,
               'title': data_train['title'][indices].values,
                'Euclidean similarity with the queried article': couple_dist[indices].ravel()})
    print("="*30,"Queried article details","="*30)
    print('title : ', data_train['title'][indices[0]])
    print('content: ', data_train['content'][indices[0]])
    print("\n","="*25,"Recommended articles : ","="*23)
    
    #return df.iloc[1:,1]
    df.sort_values(by=['Euclidean similarity with the queried article'], ascending=False, inplace=True)
    return df.iloc[:num_similar_items,]

tfidf_based_model(100, 11)

[1.41221143 1.4132776  1.41352415 ... 1.39668328 1.40614792 1.40886446]
[  100 22756  8810 10890  6777 21574 10917 11283 16386  8151  7140]
============================== Queried article details ==============================
title :  Hoãn cổ tức tới 5 lần, Sudico Sông Đà lại thất hứa với cổ đông
content:  cổ_tức sjs dời cổ_tức gom thành đợt thay_vì hoãn cổ_tức sjs sjs trường_hợp công_ty thu_xếp tiền hoạt_động_sản_xuất kinh_doanh công_ty xem_xét thanh_toán cổ_tức sudico hoãn cổ_tức hồi giao_dịch chứng_khoán tphcm hose văn_bản nhắc_nhở sjs nghiêm_túc quy_định quản_trị công_ty quy_định luật doanh_nghiệp cổ_tức thanh_toán đầy_đủ thời_hạn kết_thúc họp đhđcđ thường_niên sjs chậm_trễ chia cổ_tức quy_định quyền_lợi cổ_đông_sở nhắc_nhở sjs nhanh_chóng nghĩa_vụ cổ_tức cổ_đông quy_định hiện_hành nhắc_nhở sjs vốn chi_trả cổ_tức chủ_yếu thu dự_án nam khánh dự_án tổng công_nợ đồng đối_tác chậm_trễ thanh_toán công_ty thiếu_hụt vốn chi_trả cổ_tức sjs làm_việc đối_tác khách_hàng biện_pháp công_tác thu

,publish_date,title,Euclidean similarity with the queried article
10,23/09/19 13:14 GMT+7,Bến xe Miền Tây chi hàng trăm tỷ đồng chia cổ tức,1.227044
9,20/09/19 09:20 GMT+7,Sudico: Đến hẹn lại... khất lần cổ tức,1.216648
8,20/09/19 10:54 GMT+7,Sudico: Đến hẹn lại... khất lần cổ tức,1.216623
7,22/09/19 11:44 GMT+7,Danh sách doanh nghiệp chốt trả cổ tức tuần từ...,1.209554
6,17/09/19 19:06 GMT+7,PVS dự chi hơn 334 tỷ đồng trả cổ tức năm 2018,1.207472
5,19/09/19 11:13 GMT+7,Tập đoàn Sao Mai (ASM) dự chi gần 260 tỷ đồng ...,1.182336
4,19/09/19 11:53 GMT+7,Thủy điện – Điên lực 3 (DRL) tạm ứng cổ tức đợ...,1.123389
3,20/09/19 07:06 GMT+7,ASM tạm ứng cổ tức đợt 1/2019,1.113805
2,20/09/19 15:54 GMT+7,"Thói quen 'khất lần' khó bỏ, Sudico lại lùi ng...",0.868867
1,20/09/19 16:55 GMT+7,Sông Đà Sudico lại 'chây ì' cổ tức,0.545596


## Test with data test after lda model

In [6]:
data_test = pd.read_csv('TopicModeling/data/data_test.csv', usecols=['title','summarize','content','created','url','category','tag','source', 'group'])
data_train.head()

,title,summarize,content,created,url,category,tag,source
0,Phát động cuộc thi tiếng Anh TOEFL Challenge d...,Tổ chức Giáo dục IIG Việt Nam phát động cuộc t...,quang_cảnh phát_động tổ_chức thi ủng_hộ gdđt g...,20/09/19 10:27 GMT+7,https://baomoi.com/c/32258593.epi,Giáo dục,TOEFL Primary Challenge TOEFL Primary TOEFL Ju...,GD&TĐ
1,Hải Phòng: 4 cháu bé nhập viện cấp cứu sau khi...,Sau khi chia nhau uống chai sữa hoa quả do 1 n...,bác_sĩ phạm thị thành giám_đốc bvđk chân xác_n...,17/09/19 22:00 GMT+7,https://baomoi.com/c/32226941.epi,Đời sống,Chai sữa H.A. Công an phường Lạch Tray Ngộ độc...,GĐ&XH
2,"Văn Hậu được Heerenveen đăng ký lần đầu, có th...",Hậu vệ người Việt Nam ngồi ghế dự bị Heerenvee...,chính_thức tập_luyện heerenveen văn hậu chính_...,22/09/19 16:50 GMT+7,https://baomoi.com/c/32283331.epi,Thể thao,Lucas Woudenberg Văn Hậu Dự bị Hậu vệ Johny Ja...,VTC
3,Anh Nguyễn Tuấn Nam tái đắc cử Chủ tịch Hội LH...,"Sáng nay (20/9), Đại hội đại biểu Hội LHTNVN t...",đoàn chủ_tịch đại_hội lhtn tỉnh lạng sơn ảnh c...,20/09/19 11:06 GMT+7,https://baomoi.com/c/32259106.epi,Xã hội,Nguyễn Tuấn Nam Hội LHTNVN Nguyễn Kim Quy Hồ P...,Tiền Phong
4,Trồng hoa kiểng trong chậu phù hợp với nông ng...,Trung tâm Khuyến nông TPHCM vừa triển khai mô ...,mô_hình triển_khai hoa dừa cạn yến thảo nguồn_...,23/09/19 09:49 GMT+7,https://baomoi.com/c/32289518.epi,Kinh tế,Trung tâm Khuyến nông TPHCM Hoa kiểng Dừa cạn ...,SGGP


In [7]:
data_test.dropna(subset=['summarize'], inplace=True)
data_test.dropna(subset=['created'], inplace=True)

In [8]:
data_test[data_test['created']==None]

,title,summarize,content,created,url,category,tag,source,group


In [10]:
def tfidf_based_model(row_index, num_similar_items):
    couple_dist = pairwise_distances(tfidf_title_features, tfidf_title_features[row_index])
    print(couple_dist.ravel())
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    print(data_test['created'][indices].values)
    df = pd.DataFrame({'publish_date': data_test['created'][indices].values,
               'title': data_test['title'][indices].values,
               'group': data_test['group'][indices].values,
                'Euclidean similarity with the queried article': couple_dist[indices].ravel()})
    print("="*30,"Queried article details","="*30)
    print('title : ', data_test['title'][indices[0]])
    print('content: ', data_test['content'][indices[0]])
    group = data_test['group'][indices[0]]
    print('group: ', group)
    print("\n","="*25,"Recommended articles : ","="*23)
    
    #return df.iloc[1:,1]
    # df.sort_values(by=['Euclidean similarity with the queried article'], ascending=False, inplace=True)
    return df.iloc[:num_similar_items,df['group']==group]

tfidf_based_model(100, 11)

[1.41221143 1.4132776  1.41352415 ... 1.39668328 1.40614792 1.40886446]


KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'